In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [13]:
# Як приклад — спершу вручну скачай CSV з частиною Amazon dataset (наприклад, 1M reviews)
# Наприклад, https://path_to_your_amazon_reviews_subset.csv
# Завантажуємо датасет Amazon Polarity (містить позитивні/негативні відгуки)
ds = load_dataset("amazon_polarity")

# Беремо тільки "content" (текст відгуку) та "label" (0=негатив, 1=позитив)
texts  = ds["train"]["content"][:200_000]
labels = ds["train"]["label"][:200_000]

# Розподіл на train/test (20% на тест)
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [14]:


# Параметри
max_words = 30000
max_len   = 200
embed_dim = 64

In [15]:
# Векторизатор
vectorizer = layers.TextVectorization(
    max_tokens=max_words,
    output_mode='int',
    output_sequence_length=max_len
)
vectorizer.adapt(X_train)

In [16]:
# Модель
model = keras.Sequential([
    vectorizer,
    layers.Embedding(max_words, embed_dim, mask_zero=True),
    layers.Bidirectional(layers.LSTM(128, dropout=0.2, return_sequences=True)),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
# model.summary()

In [17]:
# Датасети tf.data
BATCH = 512
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(20000).batch(BATCH).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH).prefetch(tf.data.AUTOTUNE)


In [18]:
# Training з verbose=1 (прогрес-бар)
history = model.fit(train_ds, 
                    validation_data=test_ds, 
                    epochs=5, 
                    verbose=1)


Epoch 1/5


c:\Users\vlaad\anaconda3\Lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'global_max_pooling1d_1' (of type GlobalMaxPooling1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 578s 2s/step - accuracy: 0.8338 - loss: 0.3776 - val_accuracy: 0.8791 - val_loss: 0.2907
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 528s 2s/step - accuracy: 0.9022 - loss: 0.2491 - val_accuracy: 0.8925 - val_loss: 0.2598
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 493s 2s/step - accuracy: 0.9265 - loss: 0.1960 - val_accuracy: 0.8959 - val_loss: 0.2650
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 34514s 111s/step - accuracy: 0.9443 - loss: 0.1559 - val_accuracy: 0.8943 - val_loss: 0.2838
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 483s 2s/step - accuracy: 0.9566 - loss: 0.1262 - val_accuracy: 0.8917 - val_loss: 0.3322


In [19]:
# Оцінювання
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 45s 574ms/step - accuracy: 0.8917 - loss: 0.3322
Test loss: 0.3322, Test accuracy: 0.8917


In [30]:
def predict_texts(texts):
    if isinstance(texts, str):
        texts = [texts]
    tensor = tf.constant(texts, dtype=tf.string)
    proba = model.predict(tensor, verbose=0)
    
    for t, p in zip(texts, proba):
        prediction = float(p)
        if prediction > 0.5:
            print(f"Відгук позитивний 🙂 (ймовірність {prediction:.2f})")
        else:
            print(f"Відгук негативний 🙁 (ймовірність {1-prediction:.2f})")


In [33]:
predict_texts([
    "the movie was very interesting, exciting",
    "the film is not interesting",
     "Terrible product, stopped working after two uses",
    "Amazing build quality and super comfortable!"
])



c:\Users\vlaad\anaconda3\Lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'global_max_pooling1d_1' (of type GlobalMaxPooling1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Відгук позитивний 🙂 (ймовірність 0.94)
Відгук негативний 🙁 (ймовірність 0.97)
Відгук негативний 🙁 (ймовірність 1.00)
Відгук позитивний 🙂 (ймовірність 0.99)


C:\Users\vlaad\AppData\Local\Temp\ipykernel_3704\1597481232.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediction = float(p)
